In [ ]:
# Some imports to use in notebok
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial.distance import cdist
import scipy.spatial.distance as dist
from scipy.optimize import curve_fit
import matplotlib.patches as mpatches
import copy
import time
from tqdm import tqdm
from skimage import color
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap

In [ ]:
# Let's try a three facies approach:
### Using alternative dataset
df = pd.read_csv('C:\\Users\\Gerrit\\Desktop\\Masterarbeit\\Python\\changer\\Q565756_raw.csv',)
data = df.values
data = data[0::4]
data[:,0] = data[:,0]-data[:,0].min() # I think we should normalize the y-coordinates to be distances (in m)
y_max = data[:,0].max()
z_max = np.abs(data[:,1].min())# hier Minimum nehmen weil Tiefen negativ sind und dann Absolutwert

data_3 = copy.deepcopy(data)


data_k1 = copy.deepcopy(data_3)
data_k2 = copy.deepcopy(data_3)
data_k3 = copy.deepcopy(data_3)

In [ ]:
plt.scatter(x=data_3[:,0], y=data_3[:,1], c=data_3[:,2], cmap='viridis')
#plt.imshow(im_sub, cmap='gray', origin='lower')
plt.title('Bohrlochdaten')
plt.colorbar()

## 2. Variogram analyis
<a id="2"></a>

The first step of our geostatistical analysis is to describe the spatial correlation of our variable of interest. We do this in form of a variogram. 

In order to retrieve the variogram from our data we have to perform multiple steps. For each pointpair in our dataset we can calculate a measure of dissimilarity called *semivariance* ($\hat{\gamma}(\vec{h})$) that is dependent only on the distance between the two locations:

$$\hat{\gamma}(\vec{h}) = \frac{1}{2} \sum_{\alpha=1}^n \left(z(\vec{x_\alpha}+\vec{h}) - z(\vec{x_\alpha})\right)^2 $$

Plotting all semivariances against the so called *lag distance* ($\vec{h}$) leads to a plot called the *variogram cloud*, that is often quite messy due to the number of point pairs. To resolve this issue lag distances are binned based on a lag tolerance ($\Delta \vec{h}$) and the corresponding *semivariances* are averaged for each bin to create a plot called the *experimental varigoram*.
the customised function looks as follows:
$$\hat{\gamma}(\vec{h}) = \frac{1}{2} \sum_{\alpha=1}^n \left(z(\vec{x_\alpha}+\vec{h}) - z(\vec{x_\alpha})\right)^2*log(1+X)$$ 
In this function, X is a matrix that contains the distance in x-direction between all points.
between them looks like this:
The following function performs all the steps to create an *experimental variogram* based on a given dataset. 

In [ ]:
def full_variogram(x_pos, y_pos, vals, bin_size, start=0, stop=180):
    """Calculate experimental variogram for sampled values
    
    **Arguments**:
        - x_pos (np.array) : x-coordiantes of sampled locations
        - y_pos (np.array) : y-coordiantes of sampled locations
        - vals (np.array) : values at corresponding x-y positions
        - bin_size (float) :bin size
        - start (float) : minimum lag distance
        - stop (float) : maximum lag distance
        
    **Returns**:
        - h_bins (np.array) = average lag distances for bins
        - ave_vals (np.array) = average semivariances for bins
    """
    X = np.vstack([x_pos, y_pos]).transpose()
    
    h = dist.squareform(dist.pdist(X))
    n = len(x_pos)

    gamma = np.empty((n, n))
    
    for pos in range (n):
        gamma[pos, :] = ((vals[:]-vals[pos])**2)/2 * np.log(1 + np.abs(X[:, 0] - X[pos, 0]))
    
    # Second step: Bins and stuff
    maxim = stop-start
    bins = maxim/bin_size
    bins = int(bins)

    h_bins = np.empty(bins)
    ave_vals = np.empty(bins)

    for j in range(bins):
        # create mask
        mask = np.where((h > start) & (h < (start+bin_size)))
        # use mask
        ave_vals[j] = np.average(gamma[mask])
        h_bins[j] = (start+start+bin_size)/2
        start += bin_size
        
    ## mask if there start to be nans in the ave_vals (ich nehme an das passiert wenn die Distanz größer wird als die Distanz der Daten)
    mask = ~np.isnan(ave_vals)

    return h_bins[mask], ave_vals[mask]

In [ ]:
# Create a discrete grid to estimate on (reasonable resolution)
x = np.linspace(0, y_max,20)
y = np.linspace(-z_max,5,20)
locs = np.vstack((x,y))
locs = locs.T

xv, yv = np.meshgrid(x, y)
positions = np.vstack([xv.ravel(), yv.ravel()]).T

# just showing grid locations
plt.scatter(positions[:,0], positions[:,1])
plt.show()

In [ ]:
#defining experimental_varogramm to automatical fit the data
def exponential_variogram_model(d, range_, sill):
    '''Exponential variogram function
    
    **Arguments**:
        - d (np.array) : positions to evaluate function
        - range_ (float) : range parameter of model
        - sill (np.array) : sill parameter of model
        
    **Returns**:
        - gamma (np.array) : theoretical exponential semivariance
    '''
    nugget = 0 # Advanced 
    psill = sill- nugget
    gamma = psill * (1. - np.exp(-d / (range_))) + nugget
    return gamma

In [ ]:
# Defining three subdatasets based on indicator function

# first threshold for facies 1 
mask1 = data_k1[:,2]>0
data_k1[:,2][mask1]=0
data_k1[:,2][~mask1]=1

# second threshold for facies 2
mask2 = data_k2[:,2]==1
data_k2[:,2][mask2]=1
data_k2[:,2][~mask2]=0

# third threshold for facies 3 
mask3 = data_k3[:,2]>1
data_k3[:,2][mask3]=1
data_k3[:,2][~mask3]=0


In [ ]:
# Here we would actually require a step with a variogram for each threshold!

h_bins_k1, ave_vals_k1 = full_variogram(data_3[:,0], data_3[:,1], data_k1[:,2], 5)
h_bins_k2, ave_vals_k2 = full_variogram(data_3[:,0], data_3[:,1], data_k2[:,2], 5)
h_bins_k3, ave_vals_k3 = full_variogram(data_3[:,0], data_3[:,1], data_k3[:,2], 5)

# automatied fitting
variogram_parameters1, par_b1 = curve_fit(exponential_variogram_model, h_bins_k1, ave_vals_k1)
variogram_parameters2, par_b2 = curve_fit(exponential_variogram_model, h_bins_k2, ave_vals_k2)
variogram_parameters3, par_b3 = curve_fit(exponential_variogram_model, h_bins_k3, ave_vals_k3)

# Plotting
fig, ax = plt.subplots(1,3, figsize=(16,4))

ax[0].plot(h_bins_k1, ave_vals_k1, 'o')
ax[1].plot(h_bins_k2, ave_vals_k2, 'o')
ax[2].plot(h_bins_k3, ave_vals_k3, 'o')

# again here just eyeballing
x_temp = np.linspace(0,y_max,100)
ax[0].plot(x_temp, exponential_variogram_model(x_temp, variogram_parameters1[0], variogram_parameters1[1]))
ax[1].plot(x_temp, exponential_variogram_model(x_temp, variogram_parameters2[0], variogram_parameters2[1]))
ax[2].plot(x_temp, exponential_variogram_model(x_temp, variogram_parameters3[0], variogram_parameters3[1]))

print(variogram_parameters1)
print(variogram_parameters2)
print(variogram_parameters3)

#Plotting range parameter 
#ax[0].plot((variogram_parameters1[0], variogram_parameters1[0]), np.linspace(0, variogram_parameters1[1], 2))
#ax[1].plot((variogram_parameters2[0], variogram_parameters2[0]), np.linspace(0, variogram_parameters2[1], 2))
#ax[2].plot((variogram_parameters3[0], variogram_parameters3[0]), np.linspace(0, variogram_parameters3[1], 2))

# labeling
ax[0].set_title('Experimentelles Semivariogramm für Sand')
ax[0].set_xlabel('Lag Distanz $h$')
ax[0].set_ylabel('Semivarianz $\gamma^*$')
ax[1].set_title('Experimentelles Semivariogramm für Schluff')
ax[1].set_xlabel('Lag Distanz $h$')
ax[1].set_ylabel('Semivarianz $\gamma^*$')
ax[2].set_title('Experimentelles Semivariogramm für Ton')
ax[2].set_xlabel('Lag Distanz $h$')
ax[2].set_ylabel('Semivarianz $\gamma^*$')

In [ ]:
# perform kriging for all three thesholds
results_k1 = kriging_grid(data_k1, positions, range_=variogram_parameters1[0], sill=variogram_parameters1[1])
results_k2 = kriging_grid(data_k2, positions, range_=variogram_parameters2[0], sill=variogram_parameters2[1])
results_k3 = kriging_grid(data_k3, positions, range_=variogram_parameters3[0], sill=variogram_parameters3[1])

In [ ]:
# Plotting resultung probability maps
fig, ax = plt.subplots(1,3, figsize=(16,6))

ax[0].set_ylim(-z_max,5)
ax[0].set_xlim(5,y_max)

ax[1].set_ylim(-z_max,5)
ax[1].set_xlim(5,y_max)

ax[2].set_ylim(-z_max,5)
ax[2].set_xlim(5,y_max)

# creating the contourf kriging result
x = np.linspace(0, y_max,20)
y = np.linspace(-z_max,5,20)
X, Y = np.meshgrid(x, y)
Z1 = results_k1.reshape(X.shape)
Z2 = results_k2.reshape(X.shape)
Z3 = results_k3.reshape(X.shape)

CS1 = ax[0].contourf(X, Y, Z1, 10, cmap=plt.cm.viridis)
CS2 = ax[1].contourf(X, Y, Z2, 10, cmap=plt.cm.viridis)
CS2 = ax[2].contourf(X, Y, Z3, 10, cmap=plt.cm.viridis)

# plotting input data
#ax[0].scatter(data[:,0], data[:,1], color='black')
#ax[1].scatter(data[:,0], data[:,1], color='black')
#ax[2].scatter(data[:,0], data[:,1], color='black')

ax[0].set_aspect('equal', adjustable='box')
ax[1].set_aspect('equal', adjustable='box')
ax[2].set_aspect('equal', adjustable='box')

ax[0].title.set_text('Wahrscheinlichkeit Vorhandensein von Sand')
ax[1].title.set_text('Wahrscheinlichkeit Vorhandensein von Schluff')
ax[2].title.set_text('Wahrscheinlichkeit Vorhandensein von Ton')

cb_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])
cbar = fig.colorbar(CS1, cax=cb_ax)


plt.show()

In [ ]:
#creating structur modell
# combining results for all three facies
results_comb = np.vstack((results_k1, results_k2, results_k3)).T

# finding facies with highest probability for each location
highest_prob = np.argmax(results_comb, axis=1)

# Manuell festgelegte Farben für Sand, Schluff und Ton
custom_colors = ['#FFD700', '#04B404', '#A52A2A']

# Plotting
fig, ax = plt.subplots(figsize=(8, 8))

Z4 = highest_prob.reshape(X.shape)

# Verwenden Sie eine benutzerdefinierte Colormap mit den festgelegten Farben
custom_cmap = ListedColormap(custom_colors, name='custom_cmap', N=len(custom_colors))

# Plotting mit den zugeordneten Farben
CS4 = ax.imshow(Z4, origin='lower', cmap=custom_cmap, extent=[0, y_max, -z_max, 0], vmin=0, vmax=len(custom_colors)-1)

ax.set_aspect('equal', adjustable='box')
# Ändern Sie die Schriftgröße des Titels
ax.title.set_text('Strukturmodell')
ax.title.set_fontsize(24)  # Hier können Sie die gewünschte Schriftgröße festlegen

# Eindeutige Werte und Farben für die Legende
values = [0, 1, 2]

# Erstellen Sie Patches für die Legende mit benutzerdefinierten Labels
patches = [mpatches.Patch(color=custom_colors[value], label=" {l}".format(l=('Sand', 'Schluff', 'Ton')[value])) for value in values]

# Legende hinzufügen
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, prop={'size': 20})

plt.show()

In [ ]:
# Thermal conductivity map based on structure model

thermal_conductivity_values = [2.78, 1.83, 1.52]

# Transfer to categorical map
results_comb = np.vstack((results_k1, results_k2, results_k3)).T
highest_prob = np.argmax(results_comb, axis=1)
Z4 = highest_prob.reshape(X.shape)

# Plotting mit Wärmeleitfähigkeitswerten
fig, ax = plt.subplots(figsize=(8, 8))

# Benutzen Sie den Z-Wert, um die entsprechenden Wärmeleitfähigkeitswerte zu erhalten
thermal_conductivity_map = np.array([thermal_conductivity_values[z] for z in Z4.flatten()])

# Reshape für die Darstellung
thermal_conductivity_map = thermal_conductivity_map.reshape(Z4.shape)

# Darstellung des Wärmeleitfähigkeitsmodells
CS4 = ax.imshow(thermal_conductivity_map, origin='lower', cmap=plt.cm.plasma, extent=[0, y_max, -z_max, 0])

ax.set_aspect('equal', adjustable='box')
ax.title.set_text('Wärmeleitfähigkeitsmodell')
ax.title.set_fontsize(24)  # Hier können Sie die gewünschte Schriftgröße festlegen

# Legende für Wärmeleitfähigkeitswerte erstellen und die Größe anpassen
cbar = plt.colorbar(CS4, ax=ax, orientation='vertical', shrink=0.8)  # Sie können den shrink-Wert anpassen
cbar.set_label('Wärmeleitfähigkeitswert (in W/mK)', rotation=270, labelpad=15)

plt.show()